In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
sharks = pd.read_csv('attacks.csv')
sharks_clean = sharks.copy()

In [3]:
print(sharks_clean.shape)
sharks_clean.head()

(5992, 24)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [4]:
sharks_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
sharks_clean.columns = ['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time',
       'Species', 'Investigator_or_Source', 'pdf', 'href_formula', 'href',
       'Case_Number.1', 'Case_Number.2', 'original_order', 'Unnamed:_22',
       'Unnamed:_23']

In [6]:
nonull=sharks_clean.isnull().sum()
nonull[nonull==0]

Case_Number       0
Date              0
Year              0
Type              0
pdf               0
Case_Number.1     0
Case_Number.2     0
original_order    0
dtype: int64

In [7]:
nonull=sharks_clean.isnull().sum()
nonull[nonull>0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal_(Y/N)                 19
Time                      3213
Species                   2934
Investigator_or_Source      15
href_formula                 1
href                         3
Unnamed:_22               5991
Unnamed:_23               5990
dtype: int64

In [8]:
sharks_clean[['Age','Time','Species']].head(60)

,Age,Time,Species
0,16,13h00,NaN
1,36,11h00,NaN
2,43,10h43,NaN
3,NaN,NaN,NaN
4,NaN,NaN,2 m shark
5,NaN,NaN,NaN
6,60s,15h15,3' to 4' shark
7,51,14h30,"Tiger shark, 10?"
8,50,15h40,NaN
9,12,Late afternoon,NaN


In [9]:
sharks_clean[['Age','Time','Species']] = sharks_clean[['Age','Time','Species']].fillna('NA')

In [10]:
sharks_clean.dropna(subset=['Country','Area','Location'],how='all',inplace=True)

In [11]:
null_pais = sharks_clean[(sharks_clean['Country'].isnull()==True)]
null_pais = null_pais[['Area', 'Location']]
null_pais

,Area,Location
2731,English Channel,NaN
3162,Caribbean Sea,Between St. Kitts & Nevis
3379,NaN,Florida Strait
3792,"Between Timor & Darwin, Australia",NaN
4005,Near the Andaman & Nicobar Islands,NaN
4040,Between Comores & Madagascar,Geyser Bank
4271,Caribbean Sea,Between Cuba & Costa Rica
4412,NaN,225 miles east of Hong Kong
4473,Off South American coast,NaN
4485,300 miles east of St. Thomas (Virgin Islands),NaN


In [12]:
sharks_clean.loc[2731,'Country']='UNITED KINGDOM'
sharks_clean.loc[3162,'Country']='CARIBBEAN SEA'
sharks_clean.loc[3379,'Country']='USA'
sharks_clean.loc[3792,'Country']='AUSTRALIA'
sharks_clean.loc[4005,'Country']='BAY OF BENGAL'
sharks_clean.loc[4040,'Country']='MADAGASCAR'
sharks_clean.loc[4271,'Country']='CARIBBEAN SEA'
sharks_clean.loc[4412,'Country']='HONG KONG'
sharks_clean.loc[4473,'Country']='OCEAN'
sharks_clean.loc[5189,'Country']='OCEAN'
sharks_clean.loc[4485,'Country']='VIRGIN ISLANDS'
sharks_clean.loc[4790,'Country']='FRANCE'
sharks_clean.loc[5370,'Country']='MEDITERRANEAN SEA'
sharks_clean.loc[5558,'Country']='OCEAN'
sharks_clean.loc[5560,'Country']='SANTA CRUZ'
sharks_clean.loc[5847,'Country']='ANTIGUA'
sharks_clean.loc[5866,'Country']='IONIAN SEA'
sharks_clean.loc[5868,'Country']='IONIAN SEA'
sharks_clean.loc[5896,'Country']='BAY OF BENGAL'

In [13]:
sharks_clean[['Area','Location']] = sharks_clean[['Area','Location']].fillna('NA')

In [14]:
null_name = sharks_clean[(sharks_clean['Name'].isnull()==True)]
null_name = null_name[['Activity','Sex']]
null_name

,Activity,Sex
478,Surfing,NaN
498,NaN,M
548,Swimming,NaN
626,Diving,NaN
643,Swimming,M
...,...,...
5875,Swimming,NaN
5887,Skin diving,NaN
5908,yachtsman in a zodiac,M
5927,Swimming,NaN


In [15]:
sharks_clean.dropna(subset=['Name','Activity','Sex',],how='all',inplace=True)

In [16]:
sharks_clean[['Activity','Name','Sex']] = sharks_clean[['Activity','Name','Sex']].fillna('NA')

In [17]:
null_fatal = sharks_clean[(sharks_clean['Fatal_(Y/N)'].isnull()==True)]
null_fatal = null_fatal[['Injury','Investigator_or_Source']]
null_fatal

,Injury,Investigator_or_Source
54,"No injury, but sharks repeatedly hit their fin...","Gold Coast Bulletin, 6/4/2016"
1844,Reported as shark attack but probable drowning,"R. D. Weeks, GSAF"
2449,FATAL,"LA Times, 7/28/1989"
3280,"Diver shot the shark, then it injured his arm ...",C. Moore. GSAF
3901,Boat damaged,"C. Moore, GSAF"
4107,No injury to occupants. Shark tore nets & traw...,"C. Moore, GSAF"
4112,Human remains found in shark,"C. Moore, GSAF"
5307,"Disappeared, but shark involvement unconfirmed",C. Moore
5437,"No injury, no attack","C.Moore, GSAF"
5642,No injury,"C. Moore, GSAF"


In [18]:
sharks_clean.loc[54,'Fatal_(Y/N)']='N'
sharks_clean.loc[2449,'Fatal_(Y/N)']='Y'
sharks_clean.loc[3280,'Fatal_(Y/N)']='N'
sharks_clean.loc[4107,'Fatal_(Y/N)']='N'
sharks_clean.loc[4112,'Fatal_(Y/N)']='Y'
sharks_clean.loc[5718,'Fatal_(Y/N)']='Y'
sharks_clean.loc[5793,'Fatal_(Y/N)']='Y'

In [19]:
sharks_clean.dropna(subset=['Fatal_(Y/N)'],how='all',inplace=True)

In [20]:
sharks_clean[['Injury','Investigator_or_Source']] = sharks_clean[['Injury','Investigator_or_Source']].fillna('NA')

In [21]:
sharks_clean.dropna(subset=['href_formula','href'],how='any',inplace=True)

In [22]:
nonull=sharks_clean.isnull().sum()
nonull[nonull>0]

Unnamed:_22    5912
Unnamed:_23    5911
dtype: int64

In [23]:
sharks_clean.Type.value_counts()

Unprovoked      4351
Provoked         554
Invalid          484
Sea Disaster     217
Boat             199
Boating          108
Name: Type, dtype: int64

In [24]:
sharks_clean['Type']=sharks_clean.Type.apply(lambda x: re.sub('Boating','Boat',x))

In [25]:
sharks_clean.Type.value_counts()

Unprovoked      4351
Provoked         554
Invalid          484
Boat             307
Sea Disaster     217
Name: Type, dtype: int64

In [26]:
sharks_clean.Sex.value_counts()

M      4806
F       581
NA      521
M         2
.         1
lli       1
N         1
Name: Sex, dtype: int64

In [27]:
sharks_clean.Sex.value_counts().index

Index(['M', 'F', 'NA', 'M ', '.', 'lli', 'N'], dtype='object')

In [28]:
sharks_clean['Sex']=sharks_clean.Sex.apply(lambda x: re.sub('M ','M',x))
sharks_clean['Sex']=sharks_clean.Sex.apply(lambda x: re.sub('lli','NA',x))

In [29]:
sharks_clean.loc[4708,'Sex'] = 'NA'
sharks_clean.loc[5201,'Sex']='NA'

In [30]:
sharks_clean.Sex.value_counts()

M     4808
F      581
NA     524
Name: Sex, dtype: int64

In [31]:
sharks_clean.Species.value_counts()

NA                                          2874
White shark                                  159
Shark involvement not confirmed               79
Tiger shark                                   68
Bull shark                                    52
                                            ... 
White shark, 4.2 m [13'9"]                     1
Lemon shark, 0.5 m                             1
"small brown-colored shark"                    1
White shark, identification by K. Nakaya       1
Bronze whaler shark, 3 m [10'], 200-lb         1
Name: Species, Length: 1532, dtype: int64

In [32]:
def typew(x):
    x=x.lower()
    if 'white' in x:
        x = 'white'
    elif 'tiger' in x:
        x = 'tiger'
    elif 'bull' in x:
        x = 'bull'
    elif 'lemon' in x:
        x = 'lemon'
    elif 'mako' in x:
        x = 'mako'
    elif 'nurse' in x:
        x = 'nurse'
    elif 'cooper' in x:
        x = 'cooper'
    elif 'hammer' in x:
        x = 'hammer'
    elif 'grey' in x:
        x = 'grey'
    elif 'wobbegong' in x:
        x = 'wobbegong'
    elif 'blacktip' in x:
        x ='blacktip'
    elif 'caribbean' in x:
        x = 'caribbean'
    elif 'blue' in x:
        x = 'blue'
    elif 'dusky' in x:
        x = 'dusky'
    elif 'zambesi' in x:
        x ='zambesi'
    elif 'cocktail' in x:
        x = 'cocktail'
    elif 'zambezi' in x:
        x = 'zambesi'
    elif 'porbeagle' in x:
        x = 'porbeagle'
    elif 'bronze whaler' in x:
        x = 'bronze whaler'
    elif 'bronze' in x:
        x = 'bronze whaler'
    elif 'whale' in x:
        x = 'bronze whaler'
    elif 'raggedtooth' in x:
        x = 'raggedtooth'
    elif 'basking' in x:
        x = 'basking'
    elif 'spinner' in x:
        x = 'spinner'
    elif 'angel' in x:
        x = 'angel'
    elif 'sand' in x:
        x = 'sand'
    elif 'gill' in x:
        x = 'gill'
    elif 'galapagos' in x:
        x = 'galapagos'
    elif 'reef' in x:
        x = 'reef'
    elif 'dog' in x:
        x = 'dog'
    elif 'salmon' in x:
        x = 'salmon'
    elif 'shovelnose' in x:
        x = 'shovelnose'
    elif 'thresher' in x:
        x = 'thresher'
    elif 'barracuda' in x:
        x = 'barracuda'
    elif 'carpet' in x:
        x = 'carpet'
    elif 'silky' in x:
        x = 'silky'
    elif 'banjo' in x:
        x = 'banjo'
    elif 'leopard' in x:
        x = 'leopard'
    elif 'goblin' in x:
        x = 'goblin'
    elif 'blackfin' in x:
        x = 'blackfin'
    elif 'shovel' in x:
        x = 'shovelnose' 
    else:
        x = 'NA'
    return x

In [33]:
sharks_clean['Species']=sharks_clean.Species.apply(typew)

In [34]:
sharks_clean.Species.value_counts()

NA               4046
white             643
tiger             279
bull              166
blacktip          101
nurse              93
bronze whaler      73
blue               58
mako               54
hammer             48
wobbegong          44
raggedtooth        43
zambesi            35
lemon              31
sand               25
grey               25
spinner            20
gill               18
reef               17
caribbean          15
dusky              14
dog                10
basking             7
porbeagle           7
carpet              7
angel               6
galapagos           5
silky               4
thresher            4
shovelnose          4
salmon              3
barracuda           2
leopard             2
blackfin            1
goblin              1
banjo               1
cocktail            1
Name: Species, dtype: int64

In [38]:
sharks_clean.Country = sharks_clean.Country.astype('str')

In [58]:
sharks_clean.Country = sharks_clean.Country.str.replace(' ', '').str.replace('/', '').str.replace('?','').str.replace('(','').str.replace(')','').str.replace('.','')

In [59]:
sharks_clean.Country.value_counts()

usa               2113
australia         1273
southafrica        560
papuanewguinea     131
newzealand         121
                  ... 
ghana                1
sweden               1
curacao              1
southchinasea        1
syria                1
Name: Country, Length: 190, dtype: int64

In [60]:
sharks_clean['Country']=sharks_clean.Country.apply(lambda x: x.lower())

In [61]:
tr = list(sharks_clean.Country.value_counts().index)

In [62]:
tr

['usa',
 'australia',
 'southafrica',
 'papuanewguinea',
 'newzealand',
 'brazil',
 'bahamas',
 'mexico',
 'italy',
 'fiji',
 'philippines',
 'reunion',
 'newcaledonia',
 'mozambique',
 'cuba',
 'egypt',
 'spain',
 'india',
 'panama',
 'japan',
 'croatia',
 'solomonislands',
 'iran',
 'hongkong',
 'greece',
 'jamaica',
 'frenchpolynesia',
 'indonesia',
 'pacificocean',
 'england',
 'tonga',
 'bermuda',
 'atlanticocean',
 'vietnam',
 'marshallislands',
 'vanuatu',
 'srilanka',
 'france',
 'southatlanticocean',
 'iraq',
 'costarica',
 'venezuela',
 'turkey',
 'unitedkingdom',
 'senegal',
 'kenya',
 'canada',
 'caribbeansea',
 'yemen',
 'newguinea',
 'columbia',
 'sierraleone',
 'taiwan',
 'southkorea',
 'indianocean',
 'ecuador',
 'seychelles',
 'madagascar',
 'tanzania',
 'scotland',
 'chile',
 'samoa',
 'northpacificocean',
 'mauritius',
 'dominicanrepublic',
 'nicaragua',
 'okinawa',
 'singapore',
 'kiribati',
 'newbritain',
 'china',
 'thailand',
 'somalia',
 'israel',
 'midatlantico